### Select Twogram Threegram Fourgram Fivegram Sixgram Sevengram Eightgram According To Word Usage Limit

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 40000  # 1000 native word end index

# word usage
word_usage_limit = False
min_word_use_num = 1
max_word_use_num = 3

# sentence check
twogram_sentence_check = False  # True, False
threegram_sentence_check = False
fourgram_sentence_check = False
fivegram_sentence_check = False
sixgram_sentence_check = False
sevengram_sentence_check = False
eightgram_sentence_check = False

In [4]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [5]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [6]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 6/Result/1-Select Twogram Threegram According To Word Usage Limit"

Path(path).mkdir(parents=True, exist_ok=True)

In [7]:
#disable_video_id_list = ["H6E6N70jYqI","BoaYsdPtJYA","HUZINsU40Fk","Et3diPcEmfY"]
disable_video_id_list = []

In [8]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987921,karneleme,5
987922,karnaya,5
987923,dörtlümüzün,5
987924,karnavalınız,5


In [9]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
39995,bakacağımı,1020
39996,incelerken,1020
39997,adina,1020
39998,göndermemiz,1020


In [10]:
word_list = df_word_select["word"].to_list()
len(word_list)

40000

In [11]:
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram = df_twogram_sent.loc[:,["twogram","frequency"]]
else:
    df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")  
    df_twogram = df_twogram.loc[:,["twogram","frequency"]]

df_twogram

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
4457029,fikret cibran,3
4457030,romalı fikret,3
4457031,fikret ciooney,3
4457032,fikret cisco,3


In [12]:
df_twogram_used = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/N Gram Used/Twogram_Used.xlsx")
#df_twogram_used

In [13]:
twogram_used_list = list(set(df_twogram_used["twogram"]))

In [14]:
# Twogram Func
d_list2  = df_twogram.iloc[:,0].values.tolist()

resultlist2 = []
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def word_in_wordgroup(d_list2):
    mergelist2 = []
    try:
        word = d_list2.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist2.append(word[j])
            if len(mergelist2) == len(word):
                    resultlist2.append(d_list2)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list2) # string_word liste

result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=[0])  # add columns parameter for empty result
df_result2 = df_result2.rename(columns = {0: "twogram"})
df_merge2 = pd.merge(df_result2, df_twogram, how="left", on="twogram")
df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
df_merge_result2.drop_duplicates(inplace=True)
df_merge_result2.reset_index(drop=True, inplace=True)
df_twogram_select = df_merge_result2
df_twogram_select

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
2481721,ki sorulara,6
2481722,harika makale,6
2481723,kızın yaşadığı,6
2481724,onları korkuyla,6


In [15]:
df_twogram_select = df_twogram_select[~df_twogram_select["twogram"].isin(twogram_used_list)]
df_twogram_select

,twogram,frequency
6,hayır hayır,266457
53,burada ne,93996
55,sana bir,93343
65,kapa çeneni,79080
74,sen ne,72768
...,...,...
2481721,ki sorulara,6
2481722,harika makale,6
2481723,kızın yaşadığı,6
2481724,onları korkuyla,6


In [16]:
if word_usage_limit:
    df_twogram_word_usage = word_usage_result(word_list, df_twogram_select, "twogram", "frequency", min_word_use_num, max_word_use_num)
else:
    df_twogram_word_usage = df_twogram_select.head(200)

df_twogram_word_usage

,twogram,frequency
6,hayır hayır,266457
53,burada ne,93996
55,sana bir,93343
65,kapa çeneni,79080
74,sen ne,72768
...,...,...
418,rica ederim,21163
419,benim hatam,21141
420,çok özür,21109
421,sen misin,21097


In [17]:
word_count_result(df_twogram_word_usage, ["twogram"], set_condition=False)

,word,word_count
0,bir,27
1,ne,21
2,bu,13
3,sen,11
4,ben,9
...,...,...
196,yapacaksın,1
197,yapacağım,1
198,yapmak,1
199,şaka,1


In [18]:
df_twogram_word_usage.to_excel(f"{lang_folder.capitalize()}_Twogram_Selected_With_{word_end}_Word.xlsx", index=False)

In [19]:
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram = df_threegram_sent.loc[:,["threegram","frequency"]]
else:
    df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")  
    df_threegram = df_threegram.loc[:,["threegram","frequency"]]

df_threegram

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [20]:
df_threegram_used = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/N Gram Used/Threegram_Used.xlsx")
#df_threegram_used

In [21]:
threegram_used_list = list(set(df_threegram_used["threegram"]))

In [22]:
# Threegram Func
d_list3  = df_threegram.iloc[:,0].values.tolist()

resultlist3 = []
manager = multiprocessing.Manager()
resultlist3 = manager.list()

def word_in_wordgroup(d_list3):
    mergelist3 = []
    try:
        word = d_list3.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist3.append(word[j])
            if len(mergelist3) == len(word):
                    resultlist3.append(d_list3)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list3) # string_word liste

result_list3 = list(resultlist3)
df_result3 = pd.DataFrame(result_list3, columns=[0])  # add columns parameter for empty result
df_result3 = df_result3.rename(columns = {0: "threegram"})
df_merge3 = pd.merge(df_result3, df_threegram, how="left", on="threegram")
df_merge_result3 = df_merge3.sort_values(by="frequency", ascending=False)
df_merge_result3.drop_duplicates(inplace=True)
df_merge_result3.reset_index(drop=True, inplace=True)
df_threegram_select = df_merge_result3
df_threegram_select

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
2172508,balık yumurtası gibi,10
2172509,balık yiyen zavallı,10
2172510,balık yer misin,10
2172511,balık yemi almış,10


In [23]:
df_threegram_select = df_threegram_select[~df_threegram_select["threegram"].isin(threegram_used_list)]
df_threegram_select

,threegram,frequency
5,hayır hayır hayır,69443
10,sana bir şey,56713
19,için özür dilerim,36255
24,şey yolunda mı,31134
25,için teşekkür ederim,31066
...,...,...
2172508,balık yumurtası gibi,10
2172509,balık yiyen zavallı,10
2172510,balık yer misin,10
2172511,balık yemi almış,10


In [24]:
if word_usage_limit:
    df_threegram_word_usage = word_usage_result(word_list, df_threegram_select, "threegram", "frequency", min_word_use_num, max_word_use_num)
else:
    df_threegram_word_usage = df_threegram_select.head(200)

df_threegram_word_usage

,threegram,frequency
5,hayır hayır hayır,69443
10,sana bir şey,56713
19,için özür dilerim,36255
24,şey yolunda mı,31134
25,için teşekkür ederim,31066
...,...,...
361,tam olarak değil,7297
362,olduğunu mu düşünüyorsun,7297
363,seni sonra ararım,7284
364,bu çok önemli,7267


In [25]:
word_count_result(df_threegram_word_usage, ["threegram"], set_condition=False)

,word,word_count
0,bir,53
1,ne,47
2,şey,31
3,bu,17
4,var,17
...,...,...
231,ama,1
232,şeyim,1
233,şeyler,1
234,azından,1


In [26]:
df_threegram_word_usage.to_excel(f"{lang_folder.capitalize()}_Threegram_Selected_With_{word_end}_Word.xlsx", index=False)

In [27]:
if fourgram_sentence_check:
    df_fourgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Four_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fourgram_sent.rename(columns={"four_gram":"fourgram"}, inplace=True)  # ext. Not: Four_Gram_And_Sentence.csv convert to Four_Gram_And_Sentence_All.csv
    df_fourgram = df_fourgram_sent.loc[:,["fourgram","frequency"]]
else:
    df_fourgram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fourgram_Merge.csv")  
    df_fourgram = df_fourgram.loc[:,["fourgram","frequency"]]

df_fourgram

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
3052209,fikret miloya merhaba de,5
3052210,fikret millsin oğlu mu,5
3052211,fikret millet iyi iş,5
3052212,fikret millet bize bakıyor,5


In [28]:
df_fourgram_used = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/N Gram Used/Fourgram_Used.xlsx")
#df_fourgram_used

In [29]:
fourgram_used_list = list(set(df_fourgram_used["fourgram"]))

In [30]:
# Fourgram Func
d_list4  = df_fourgram.iloc[:,0].values.tolist()

resultlist4 = []
manager = multiprocessing.Manager()
resultlist4 = manager.list()

def word_in_wordgroup(d_list4):
    mergelist4 = []
    try:
        word = d_list4.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist4.append(word[j])
            if len(mergelist4) == len(word):
                    resultlist4.append(d_list4)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list4) # string_word liste

result_list4 = list(resultlist4)
df_result4 = pd.DataFrame(result_list4, columns=[0])  # add columns parameter for empty result
df_result4 = df_result4.rename(columns = {0: "fourgram"})
df_merge4 = pd.merge(df_result4, df_fourgram, how="left", on="fourgram")
df_merge_result4 = df_merge4.sort_values(by="frequency", ascending=False)
df_merge_result4.drop_duplicates(inplace=True)
df_merge_result4.reset_index(drop=True, inplace=True)
df_fourgram_select = df_merge_result4
df_fourgram_select

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
1887081,sanki bebek bile ona,7
1887082,kutsal su kana dönüştü,7
1887083,kutsal su işe yaramaz,7
1887084,merak etmeyin ben onu,7


In [31]:
df_fourgram_select = df_fourgram_select[~df_fourgram_select["fourgram"].isin(fourgram_used_list)]
df_fourgram_select

,fourgram,frequency
12,hayır hayır hayır hayır,11959
23,bir iki üç dört,7247
40,de öyle değil mi,5379
43,söylemem gereken bir şey,5219
56,benim için bir şey,4714
...,...,...
1887081,sanki bebek bile ona,7
1887082,kutsal su kana dönüştü,7
1887083,kutsal su işe yaramaz,7
1887084,merak etmeyin ben onu,7


In [32]:
if word_usage_limit:
    df_fourgram_word_usage = word_usage_result(word_list, df_fourgram_select, "fourgram", "frequency", min_word_use_num, max_word_use_num)
else:
    df_fourgram_word_usage = df_fourgram_select.head(200)
    
df_fourgram_word_usage

,fourgram,frequency
12,hayır hayır hayır hayır,11959
23,bir iki üç dört,7247
40,de öyle değil mi,5379
43,söylemem gereken bir şey,5219
56,benim için bir şey,4714
...,...,...
317,olan ne biliyor musun,1772
318,bu acil bir durum,1771
319,acil bir durum var,1768
320,ama ne yazık ki,1764


In [33]:
word_count_result(df_fourgram_word_usage, ["fourgram"], set_condition=False)

,word,word_count
0,bir,80
1,şey,51
2,ne,31
3,var,26
4,bu,24
...,...,...
235,diyeceğim,1
236,üzgünüm,1
237,özledim,1
238,altında,1


In [34]:
df_fourgram_word_usage.to_excel(f"{lang_folder.capitalize()}_Fourgram_Selected_With_{word_end}_Word.xlsx", index=False)

In [35]:
if fivegram_sentence_check:
    df_fivegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Five_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fivegram_sent.rename(columns={"five_gram":"fivegram"}, inplace=True)  # ext. Not: Five_Gram_And_Sentence.csv convert to Five_Gram_And_Sentence_All.csv
    df_fivegram = df_fivegram_sent.loc[:,["fivegram","frequency"]]
else:
    df_fivegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fivegram_Merge.csv")  
    df_fivegram = df_fivegram.loc[:,["fivegram","frequency"]]

df_fivegram

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
1096831,peder fikret hep şöyle söylerdi,4
1096832,peder fikret intihar etmeye çalışıyor,4
1096833,fikret dolson 12 gün yaşadı,4
1096834,ama fikret diye biri yoktu,4


In [36]:
df_fivegram_used = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/N Gram Used/Fivegram_Used.xlsx")
#df_fivegram_used

In [37]:
fivegram_used_list = list(set(df_fivegram_used["fivegram"]))

In [38]:
# Fivegram Func
d_list5  = df_fivegram.iloc[:,0].values.tolist()

resultlist5 = []
manager = multiprocessing.Manager()
resultlist5 = manager.list()

def word_in_wordgroup(d_list5):
    mergelist5 = []
    try:
        word = d_list5.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist5.append(word[j])
            if len(mergelist5) == len(word):
                    resultlist5.append(d_list5)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list5) # string_word liste

result_list5 = list(resultlist5)
df_result5 = pd.DataFrame(result_list5, columns=[0])  # add columns parameter for empty result
df_result5 = df_result5.rename(columns = {0: "fivegram"})
df_merge5 = pd.merge(df_result5, df_fivegram, how="left", on="fivegram")
df_merge_result5 = df_merge5.sort_values(by="frequency", ascending=False)
df_merge_result5.drop_duplicates(inplace=True)
df_merge_result5.reset_index(drop=True, inplace=True)
df_fivegram_select = df_merge_result5
df_fivegram_select

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
577651,zamanda bu teknoloji insanların günlük,10
577652,mu düşünüyorsun yoksa öyle mi,10
577653,başlayınca su olunca göreceksin ne,10
577654,o zaman nasıl bir izlenim,10


In [39]:
df_fivegram_select = df_fivegram_select[~df_fivegram_select["fivegram"].isin(fivegram_used_list)]
df_fivegram_select

,fivegram,frequency
12,bir iki üç dört beş,4477
14,hayır hayır hayır hayır hayır,4380
17,sana söylemem gereken bir şey,3822
27,bunun iyi bir fikir olduğunu,2557
29,beş dört üç iki bir,2549
...,...,...
577651,zamanda bu teknoloji insanların günlük,10
577652,mu düşünüyorsun yoksa öyle mi,10
577653,başlayınca su olunca göreceksin ne,10
577654,o zaman nasıl bir izlenim,10


In [40]:
if word_usage_limit:
    df_fivegram_word_usage = word_usage_result(word_list, df_fivegram_select, "fivegram", "frequency", min_word_use_num, max_word_use_num)
else:
    df_fivegram_word_usage = df_fivegram_select.head(200)
    
df_fivegram_word_usage

,fivegram,frequency
12,bir iki üç dört beş,4477
14,hayır hayır hayır hayır hayır,4380
17,sana söylemem gereken bir şey,3822
27,bunun iyi bir fikir olduğunu,2557
29,beş dört üç iki bir,2549
...,...,...
255,bir daha böyle bir şey,740
256,sana bir sır vereyim mi,739
257,senin için bir işim var,739
258,başka bir şey söyledi mi,737


In [41]:
word_count_result(df_fivegram_word_usage, ["fivegram"], set_condition=False)

,word,word_count
0,bir,107
1,şey,71
2,var,34
3,ne,25
4,mi,20
...,...,...
288,yolu,1
289,yargıç,1
290,çıkanı,1
291,zamana,1


In [42]:
df_fivegram_word_usage.to_excel(f"{lang_folder.capitalize()}_Fivegram_Selected_With_{word_end}_Word.xlsx", index=False)

In [43]:
if sixgram_sentence_check:
    df_sixgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Six_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_sixgram_sent.rename(columns={"six_gram":"sixgram"}, inplace=True)  # ext. Not: Six_Gram_And_Sentence.csv convert to Six_Gram_And_Sentence_All.csv
    df_sixgram = df_sixgram_sent.loc[:,["sixgram","frequency"]]
else:
    df_sixgram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Sixgram_Merge.csv")  
    df_sixgram = df_sixgram.loc[:,["sixgram","frequency"]]

df_sixgram

,sixgram,frequency
0,sana söylemem gereken bir şey var,3646
1,bir iki üç dört beş altı,2025
2,söylemek istediğin bir şey var mı,1794
3,daha iyi bir fikrin var mı,1792
4,bunun iyi bir fikir olduğunu sanmıyorum,1743
...,...,...
8421491,hatırım için bir dakika oyalanmanı sorun,3
8421492,ben onu bir dikiş makinesinin fotoğrafına,3
8421493,hazine avı senin için hâlâ bir,3
8421494,hazırlık maçları ve normal sezon futbol,3


In [44]:
df_sixgram_used = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/N Gram Used/Sixgram_Used.xlsx")
#df_sixgram_used

In [45]:
sixgram_used_list = list(set(df_sixgram_used["sixgram"]))

In [46]:
# Sixgram Func
d_list6  = df_sixgram.iloc[:,0].values.tolist()

resultlist6 = []
manager = multiprocessing.Manager()
resultlist6 = manager.list()

def word_in_wordgroup(d_list6):
    mergelist6 = []
    try:
        word = d_list6.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist6.append(word[j])
            if len(mergelist6) == len(word):
                    resultlist6.append(d_list6)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list6) # string_word liste

result_list6 = list(resultlist6)
df_result6 = pd.DataFrame(result_list6, columns=[0])  # add columns parameter for empty result
df_result6 = df_result6.rename(columns = {0: "sixgram"})
df_merge6 = pd.merge(df_result6, df_sixgram, how="left", on="sixgram")
df_merge_result6 = df_merge6.sort_values(by="frequency", ascending=False)
df_merge_result6.drop_duplicates(inplace=True)
df_merge_result6.reset_index(drop=True, inplace=True)
df_sixgram_select = df_merge_result6
df_sixgram_select

,sixgram,frequency
0,sana söylemem gereken bir şey var,3646
1,bir iki üç dört beş altı,2025
2,söylemek istediğin bir şey var mı,1794
3,daha iyi bir fikrin var mı,1792
4,bunun iyi bir fikir olduğunu sanmıyorum,1743
...,...,...
3862723,ve babamın onunla bir ilişkisi olmuştu,3
3862724,önce herhangi bir hastalık nöbeti geçirmiş,3
3862725,sene bana ağırlık seti getirmesini istemiştim,3
3862726,için bunu görmek çok iyi bir,3


In [47]:
df_sixgram_select = df_sixgram_select[~df_sixgram_select["sixgram"].isin(sixgram_used_list)]
df_sixgram_select

,sixgram,frequency
1,bir iki üç dört beş altı,2025
5,baba oğul ve kutsal ruh adına,1537
6,iki üç dört beş altı yedi,1486
7,daha önce hiç böyle bir şey,1476
8,için yapabileceğim bir şey var mı,1320
...,...,...
3862723,ve babamın onunla bir ilişkisi olmuştu,3
3862724,önce herhangi bir hastalık nöbeti geçirmiş,3
3862725,sene bana ağırlık seti getirmesini istemiştim,3
3862726,için bunu görmek çok iyi bir,3


In [48]:
if word_usage_limit:
    df_sixgram_word_usage = word_usage_result(word_list, df_sixgram_select, "sixgram", "frequency", min_word_use_num, max_word_use_num)
else:
    df_sixgram_word_usage = df_sixgram_select.head(200)
    
df_sixgram_word_usage

,sixgram,frequency
1,bir iki üç dört beş altı,2025
5,baba oğul ve kutsal ruh adına,1537
6,iki üç dört beş altı yedi,1486
7,daha önce hiç böyle bir şey,1476
8,için yapabileceğim bir şey var mı,1320
...,...,...
230,ben de sana aynı şeyi soracaktım,173
231,bu işte bir bit yeniği var,173
232,bunun senin için ne kadar zor,173
233,söylemem gereken önemli bir şey var,173


In [49]:
word_count_result(df_sixgram_word_usage, ["sixgram"], set_condition=False)

,word,word_count
0,bir,117
1,şey,67
2,var,47
3,kadar,36
4,için,30
...,...,...
309,vermeyeceğim,1
310,when,1
311,yakında,1
312,ruh,1


In [50]:
df_sixgram_word_usage.to_excel(f"{lang_folder.capitalize()}_Sixgram_Selected_With_{word_end}_Word.xlsx", index=False)

In [51]:
if sevengram_sentence_check:
    df_sevengram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Seven_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_sevengram_sent.rename(columns={"seven_gram":"sevengram"}, inplace=True)  # ext. Not: Seven_Gram_And_Sentence.csv convert to Seven_Gram_And_Sentence_All.csv
    df_sevengram = df_sevengram_sent.loc[:,["sevengram","frequency"]]
else:
    df_sevengram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Sevengram_Merge.csv")  
    df_sevengram = df_sevengram.loc[:,["sevengram","frequency"]]

df_sevengram

,sevengram,frequency
0,bir iki üç dört beş altı yedi,1455
1,iki üç dört beş altı yedi sekiz,1080
2,bana söylemek istediğin bir şey var mı,752
3,yedi altı beş dört üç iki bir,596
4,sekiz yedi altı beş dört üç iki,551
...,...,...
4775971,bilirsin o filmle ilgili bir çok iyi,3
4775972,evlenecekse bunu bilmeye hakkı var değil mi,3
4775973,evlendirmek istediği çocuğu beğenmediysen o za...,3
4775974,şeyleri tamir ettirmek için sürekli beni döverdi,3


In [52]:
df_sevengram_used = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/N Gram Used/Sevengram_Used.xlsx")
#df_sevengram_used

In [53]:
sevengram_used_list = list(set(df_sevengram_used["sevengram"]))

In [54]:
# Sevengram Func
d_list7  = df_sevengram.iloc[:,0].values.tolist()

resultlist7 = []
manager = multiprocessing.Manager()
resultlist7 = manager.list()

def word_in_wordgroup(d_list7):
    mergelist7 = []
    try:
        word = d_list7.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist7.append(word[j])
            if len(mergelist7) == len(word):
                    resultlist7.append(d_list7)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list7) # string_word liste

result_list7 = list(resultlist7)
df_result7 = pd.DataFrame(result_list7, columns=[0])  # add columns parameter for empty result
df_result7 = df_result7.rename(columns = {0: "sevengram"})
df_merge7 = pd.merge(df_result7, df_sevengram, how="left", on="sevengram")
df_merge_result7 = df_merge7.sort_values(by="frequency", ascending=False)
df_merge_result7.drop_duplicates(inplace=True)
df_merge_result7.reset_index(drop=True, inplace=True)
df_sevengram_select = df_merge_result7
df_sevengram_select

,sevengram,frequency
0,bir iki üç dört beş altı yedi,1455
1,iki üç dört beş altı yedi sekiz,1080
2,bana söylemek istediğin bir şey var mı,752
3,yedi altı beş dört üç iki bir,596
4,sekiz yedi altı beş dört üç iki,551
...,...,...
1930678,çünkü bu gezegendeki en güzel kadın benim,3
1930679,sonuçları çok garip çünkü hiç farklı birşey,3
1930680,yıl sonu ödül töreni listesi olduğunu düşündüm,3
1930681,sahibi olan bir kızla işi pişirmek senin,3


In [55]:
if word_usage_limit:
    df_sevengram_word_usage = word_usage_result(word_list, df_sevengram_select, "sevengram", "frequency", min_word_use_num, max_word_use_num)
else:
    df_sevengram_word_usage = df_sevengram_select.head(300)
    
df_sevengram_word_usage

,sevengram,frequency
0,bir iki üç dört beş altı yedi,1455
1,iki üç dört beş altı yedi sekiz,1080
2,bana söylemek istediğin bir şey var mı,752
3,yedi altı beş dört üç iki bir,596
4,sekiz yedi altı beş dört üç iki,551
...,...,...
295,yıldız sistemine girdiği andan itibaren amacı ...,64
296,fakat dünyanın ona en çok ihtiyaç duyduğu,64
297,bunun ne kadar tehlikeli olduğunu biliyor musun,64
298,ölümünden önce söylediği sözler insanları deni...,64


In [56]:
word_count_result(df_sevengram_word_usage, ["sevengram"], set_condition=False)

,word,word_count
0,bir,144
1,şey,67
2,var,52
3,için,43
4,kadar,40
...,...,...
610,süre,1
611,b,1
612,sırlar,1
613,ağlayacak,1


In [57]:
df_sevengram_word_usage.to_excel(f"{lang_folder.capitalize()}_Sevengram_Selected_With_{word_end}_Word.xlsx", index=False)

In [11]:
if eightgram_sentence_check:
    df_eightgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Eight_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_eightgram_sent.rename(columns={"eight_gram":"eightgram"}, inplace=True)  # ext. Not: Eight_Gram_And_Sentence.csv convert to Eight_Gram_And_Sentence_All.csv
    df_eightgram = df_eightgram_sent.loc[:,["eightgram","frequency"]]
else:
    df_eightgram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Eightgram_Merge.csv")  
    df_eightgram = df_eightgram.loc[:,["eightgram","frequency"]]

df_eightgram

,eightgram,frequency
0,bir iki üç dört beş altı yedi sekiz,1060
1,hem de tek bir başıma sinirlenmişim var mı,538
2,de tek bir başıma sinirlenmişim var mı kaşınan,538
3,dokuz sekiz yedi altı beş dört üç iki,513
4,sekiz yedi altı beş dört üç iki bir,497
...,...,...
2451624,daha hatanızı göreyim ikinizin kıçını da elime...,3
2451625,gerek yok bayan ama nakaratı bizimle beraber s...,3
2451626,yemeğe çıkarmanı ve karşılığında hiçbir şey be...,3
2451627,ama yalnızca cereyan yaptığı ve pantolonum olm...,3


In [12]:
# Eightgram Func
d_list8  = df_eightgram.iloc[:,0].values.tolist()

resultlist8 = []
manager = multiprocessing.Manager()
resultlist8 = manager.list()

def word_in_wordgroup(d_list8):
    mergelist8 = []
    try:
        word = d_list8.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist8.append(word[j])
            if len(mergelist8) == len(word):
                    resultlist8.append(d_list8)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list8) # string_word liste

result_list8 = list(resultlist8)
df_result8 = pd.DataFrame(result_list8, columns=[0])  # add columns parameter for empty result
df_result8 = df_result8.rename(columns = {0: "eightgram"})
df_merge8 = pd.merge(df_result8, df_eightgram, how="left", on="eightgram")
df_merge_result8 = df_merge8.sort_values(by="frequency", ascending=False)
df_merge_result8.drop_duplicates(inplace=True)
df_merge_result8.reset_index(drop=True, inplace=True)
df_eightgram_select = df_merge_result8
df_eightgram_select

,eightgram,frequency
0,bir iki üç dört beş altı yedi sekiz,1060
1,dokuz sekiz yedi altı beş dört üç iki,513
2,sekiz yedi altı beş dört üç iki bir,497
3,iki üç dört beş altı yedi sekiz dokuz,411
4,üç dört beş altı yedi sekiz dokuz on,329
...,...,...
873866,beni buraya getirmek için bu kadar zahmete gir...,3
873867,kutu gibi ama iyi kafayı mı yedin sen,3
873868,bir çizik bir şey değil ama çılgınca kanıyor,3
873869,kutu gönderdik hepsini geri almalıyız ve depoy...,3


In [13]:
if word_usage_limit:
    df_eightgram_word_usage = word_usage_result(word_list, df_eightgram_select, "eightgram", "frequency", min_word_use_num, max_word_use_num)
else:
    df_eightgram_word_usage = df_eightgram_select.head(300)
    
df_eightgram_word_usage

,eightgram,frequency
0,bir iki üç dört beş altı yedi sekiz,1060
1,dokuz sekiz yedi altı beş dört üç iki,513
2,sekiz yedi altı beş dört üç iki bir,497
3,iki üç dört beş altı yedi sekiz dokuz,411
4,üç dört beş altı yedi sekiz dokuz on,329
...,...,...
495,bence senden daha iyi bir özel ajan oldum,35
496,şeyi olduğu gibi bırakın ve bırakın sözleriniz...,35
497,piyasada arabaları çabucak en yüksek fiyata el...,35
498,dostum işinizin ehli olduğunuzu biliyor ve say...,35


In [14]:
word_count_result(df_eightgram_word_usage, ["eightgram"], set_condition=False)

,word,word_count
0,bir,196
1,ve,97
2,bu,79
3,için,72
4,şey,52
...,...,...
1323,şansını,1
1324,şanslıyız,1
1325,ümit,1
1326,özellikle,1


In [15]:
df_eightgram_word_usage.to_excel(f"{lang_folder.capitalize()}_Eightgram_Selected_With_{word_end}_Word.xlsx", index=False)

#### Copy Move And Delete

In [16]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Selected_With_{word_end}_Word.xlsx")
output_file

['Turkish_Eightgram_Selected_With_40000_Word.xlsx']

In [17]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [18]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass